In [ ]:
import os
import sys
sys.path.append('/users/vesalaia/.local/lib/python3.9/site-packages')
sys.path.append('/users/vesalaia/.local/lib/python3.9/site-packages/bin')
sys.path.append('/users/vesalaia/.local/lib/python3.9/site-packages/lib/python3.9/site-packages')

In [ ]:
import detectron2


In [ ]:
detectron2.__version__

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor

from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.utils.visualizer import ColorMode

In [ ]:
from detectron2.structures import BoxMode

In [ ]:
import torch

from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
def convert_from_cv2_to_image(img: np.ndarray) -> Image:
    # return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    return Image.fromarray(img)


def convert_from_image_to_cv2(img: Image) -> np.ndarray:
    # return cv2.cvtColor(numpy.array(img), cv2.COLOR_RGB2BGR)
    return np.asarray(img)

In [ ]:
#Elementclasses = {'Text':0,"Title":1,"List":2, "Table":3,"Figure":4}
Elementclasses = {"Illustration":0}

UNKNOWN = len(Elementclasses)-1


ElementclassLabels = [x for x in Elementclasses.keys()]
reverse_Elementclass = {v:k for k,v in Elementclasses.items()}

In [ ]:
def get_key(l):
    return reverse_Elementclass[l]

In [ ]:
config_file_path = "/scratch/project_2005488/DHH23/config.yaml"

num_classes = len(ElementclassLabels)
device = "cuda"
output_dir = "/scratch/project_2005488/DHH23/model"

def get_train_cfg(config_file_path, num_classes, device, output_dir):

    
    cfg = get_cfg()
    cfg.merge_from_file(config_file_path)
    cfg.MODEL.WEIGHTS = "/scratch/project_2005488/DHH23/model/model_final.pth"
    
    cfg.DETECTIONS_PER_IMAGE = 100

    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.MODEL.DEVICE = device
    cfg.OUTPUT_DIR = output_dir
    cfg.MASK_FORMAT = "bitmask"
    return cfg


In [ ]:
cfg = get_train_cfg(config_file_path,  num_classes, device, output_dir)

In [ ]:
def on_image(image_path, predictor):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1], metadata = {}, scale=0.5, instance_mode = ColorMode.SEGMENTATION)
    v = v.draw_instance_predictions(outputs['instances'].to("cpu"))
    plt.figure(figsize=(10,6))
    plt.imshow(v.get_image())
    plt.show()

In [ ]:
def drawBoxes(image_path, predictor):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    v = Visualizer(
        im[:, :, ::-1], 
        metadata={}, 
        scale=0.5,
        )
    for box,l,sc in zip(outputs["instances"].pred_boxes.to('cpu'),outputs["instances"].pred_classes.to('cpu'),outputs["instances"].scores.to('cpu')) :
        
        if sc >= 0.5 and l in [0,1,2]:
            v.draw_box(box)
            v.draw_text(get_key(l.item()), tuple(box[:2].numpy()))
    v = v.get_output()
    plt.figure(figsize=(10,6))
    plt.imshow(v.get_image())
    plt.show()

In [ ]:
from detectron2.engine import DefaultPredictor

cfg.MODEL.ROI_HEADS.SCORE_TRESH_TEST = 0.8
cfg.MODEL.WEIGHTS = "/scratch/project_2005488/DHH23/model/model_final_1cl.pth"
predictor = DefaultPredictor(cfg)

In [ ]:
def save_image(image, file_path):
    try:
        image.save(file_path)
    except Exception as e:
        print(f"An error occurred while saving the image: {e}")

In [ ]:
def cropBbox(img_name, bbox, label, out_dir, crop_name):
    crop_folder = os.path.join(out_dir, label)
    if not os.path.exists(crop_folder):
        try:
            os.makedirs(crop_folder)
        except OSError as e:
            print(f"An error occurred while creating the directory '{crop_folder}': {e}")

    img = Image.open(img_name).convert("RGB")
    x1, y1 = bbox[0]
    x2, y2 = bbox[1]
    if (x2>x1) and (y2>y1):
#        print('##  box=[{},{},{},{}]'.format(x1,y1,x2,y2))
        cropped_image = img.crop((x1,y1, x2,y2))
        save_image(cropped_image, os.path.join(crop_folder, crop_name))
#        print("Cropped illustration saved: {}".format(os.path.join(crop_folder, crop_name)))
#        plt.imshow(cropped_image)
#        plt.show()
        

In [ ]:
import copy
def plotBBoxes(imgpath, boxes):
    x = cv2.imread(imgpath)
    color = (255, 0, 0)
    thickness = 4
    image = copy.deepcopy(x)
    for bb in boxes:
    
        start_point = bb[0]
        end_point = bb[1]
        image = cv2.rectangle(image, start_point, end_point, color, thickness)

    plt.imshow(image) 
    plt.show()

In [ ]:
def get_regions(imagepath, model, Elementclasses):   

    reverse_class = {v:k for k,v in Elementclasses.items()}
    
    im = cv2.imread(imagepath)

    outputs = model(im)
    scores = [x.item() for x in list(outputs['instances'].scores)]
    boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in outputs['instances'].pred_boxes]
    labels = [reverse_class[i.item()] for i in outputs['instances'].pred_classes]

    return boxes, labels, scores

In [ ]:
crop_dir = "/scratch/project_2005488/DHH23/"
imagedir = "/scratch/project_2005488/DHH23/Detection"
imagelist = os.listdir(imagedir)

for imgname in imagelist:
    if imgname.endswith(".png") or imgname.endswith(".jpg") or imgname.endswith(".jpeg") or imgname.endswith(".TIF"):
#        print(imgname)
        image_path = os.path.join(imagedir,imgname)

        drawBoxes(image_path, predictor)

In [ ]:
crop_dir = "/scratch/project_2005488/DHH23/cropped"
imagedir = "/scratch/project_2005488/DHH23/Detection"
imagelist = os.listdir(imagedir)

for imgname in imagelist:
    if imgname.endswith(".png") or imgname.endswith(".jpg") or imgname.endswith(".jpeg") or imgname.endswith(".TIF"):
#        print(imgname)
        image_path = os.path.join(imagedir,imgname)
        r_boxes, r_labels, r_scores = get_regions(image_path, predictor, Elementclasses)
        basename = image_path.split("/")[-1]
        basename = basename.split(".")[0]
        i = 0
        for box,label in zip(r_boxes, r_labels):
            crop_name = basename + "_" + str(i) + ".png"
            cropBbox(image_path, box, label, crop_dir, crop_name)
            i+=1
